In [53]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 

In [54]:
dataset = pd.read_csv('bank-full.csv',header=0,sep=';')
X = dataset.iloc[:, :-1].values
Y = dataset.iloc[:, -1].values
dataset.dropna()
dataset.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


In [32]:
dataset['y'].value_counts()

no     39922
yes     5289
Name: y, dtype: int64

In [55]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
dataset['y'] = le.fit_transform(dataset['y'])

In [6]:
dataset.groupby('marital').mean()


,age,balance,day,duration,campaign,pdays,previous,y
marital,,,,,,,,
divorced,45.782984,1178.872287,15.796428,262.517188,2.630882,41.001728,0.551373,0.119455
married,43.408099,1425.925590,15.854487,253.412765,2.842875,37.950467,0.556552,0.101235
single,33.703440,1301.497654,15.708210,266.497967,2.649805,44.652385,0.642690,0.149492


In [7]:
dataset.groupby('education').mean()

,age,balance,day,duration,campaign,pdays,previous,y
education,,,,,,,,
primary,45.865567,1250.949934,15.421398,255.933002,2.834331,36.081740,0.489272,0.086265
secondary,39.964270,1154.880786,15.759159,258.685760,2.700802,42.353504,0.567753,0.105594
tertiary,39.593640,1758.416435,16.086535,258.518532,2.825577,39.024134,0.661980,0.150064
unknown,44.510501,1526.754443,15.810985,257.313947,2.849219,36.856220,0.488422,0.135703


In [56]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1,2,3,4,6,7,8,10,15])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

In [9]:
print(X)
print(len(X))

[[0.0 0.0 0.0 ... 1 -1 0]
 [0.0 0.0 0.0 ... 1 -1 0]
 [0.0 0.0 1.0 ... 1 -1 0]
 ...
 [0.0 0.0 0.0 ... 5 184 3]
 [0.0 1.0 0.0 ... 4 -1 0]
 [0.0 0.0 1.0 ... 2 188 11]]
45211


In [57]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=0)
scaler = preprocessing.StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [43]:
print(len(X_train))
print(X_test.shape[1])

31647
51


In [58]:
X1_train = pd.DataFrame(X_train)
columns=X1_train.columns
print(X1_train)
print(columns)

             0         1         2         3         4         5         6   \
0     -0.357369 -0.523580 -0.185522 -0.164113  1.934015 -0.228854 -0.190744   
1     -0.357369  1.909926 -0.185522 -0.164113 -0.517059 -0.228854 -0.190744   
2      2.798228 -0.523580 -0.185522 -0.164113 -0.517059 -0.228854 -0.190744   
3     -0.357369 -0.523580 -0.185522 -0.164113  1.934015 -0.228854 -0.190744   
4     -0.357369 -0.523580 -0.185522 -0.164113 -0.517059 -0.228854 -0.190744   
...         ...       ...       ...       ...       ...       ...       ...   
31642 -0.357369 -0.523580 -0.185522 -0.164113  1.934015 -0.228854 -0.190744   
31643 -0.357369 -0.523580 -0.185522 -0.164113  1.934015 -0.228854 -0.190744   
31644 -0.357369 -0.523580 -0.185522 -0.164113  1.934015 -0.228854 -0.190744   
31645 -0.357369 -0.523580 -0.185522 -0.164113 -0.517059  4.369592 -0.190744   
31646 -0.357369  1.909926 -0.185522 -0.164113 -0.517059 -0.228854 -0.190744   

            7         8         9   ...        41  

In [59]:
from imblearn.over_sampling import SMOTE
os = SMOTE(random_state=0)

os_data_X,os_data_y=os.fit_resample(X_train, Y_train)
os_data_X = pd.DataFrame(data=os_data_X)
os_data_y= pd.DataFrame(data=os_data_y)

os_data_y[0].value_counts()
os_data_X[1].mean()
print(len(os_data_X))

55906


In [60]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE
lr = LogisticRegression()
rfe = RFE(estimator=lr, n_features_to_select=15)
rfe = rfe.fit(os_data_X, os_data_y)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

In [61]:
print(rfe.support_)
print(rfe.ranking_)
cols = [15,17,19,20,21,22,23,25,28,29,31,33,38,42,43]
os_data_X = os_data_X[cols]
print(len(os_data_X))
print(os_data_X.shape[1])

[False False False False False False False False False False False False
 False False False  True False False False False False  True  True False
  True  True  True  True False  True False False  True  True False  True
 False  True False False False False  True False False False False  True
  True False False]
[14 24 18 17 29  7 23 25 15 16 36 37 26 21  6  1 33 13 34 32 30  1  1  2
  1  1  1  1 10  1 11 12  1  1  3  1 27  1  4  5 20 35  1  8 22 19 31  1
  1  9 28]
55906
15


In [62]:
from sklearn import metrics
lr.fit(os_data_X, os_data_y)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression()

In [63]:
X_test = pd.DataFrame(data=X_test)
print(len(X_test))
print(X_test.shape[1])
X_test = X_test[cols]
print(len(X_test))
print(X_test.shape[1])


13564
51
13564
15


In [64]:
y_pred = lr.predict(X_test)
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(lr.score(X_test, Y_test)))

Accuracy of logistic regression classifier on test set: 0.73


In [65]:
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(Y_test, y_pred)
print(confusion_matrix)

[[9029 2940]
 [ 660  935]]


In [66]:
from sklearn.metrics import classification_report
print(classification_report(Y_test, y_pred))

              precision    recall  f1-score   support

          no       0.93      0.75      0.83     11969
         yes       0.24      0.59      0.34      1595

    accuracy                           0.73     13564
   macro avg       0.59      0.67      0.59     13564
weighted avg       0.85      0.73      0.78     13564



In [71]:
Z = [25,'student','single','secondary','no',331,'no','no','telephone',26,'aug',170,4,-1,0,'unknown']
Z = pd.DataFrame(Z)
Z= Z.transpose()
Z=np.array(ct.transform(Z))
Z=scaler.transform(Z)
Z= pd.DataFrame(Z)
Z=Z[cols]
print(Z)

         15        17        19        20       21       22        23  \
0 -0.422214 -0.646483  0.135068 -0.135068  1.11992 -1.11992  0.437589   

         25        28        29        31        33       38        42  \
0 -1.356777 -0.261529  2.498375 -0.247882 -0.425067 -0.12747 -0.185157   

         43  
0  0.472456  


In [72]:
lr.predict(Z)

array(['no'], dtype=object)